In [31]:
# Dependencies and Setup
import pandas as pd


# Raw data file
file_to_load = "Resources/nfl_data.csv"

# Read purchasing file and store into pandas data frame
nfldata_raw = pd.read_csv(file_to_load)

In [34]:
#Grabbing Columns relavent to our research
nfl = nfldata_raw[["Date", "GameID", "down", "Yards.Gained", "FirstDown", "Touchdown", "PlayType", \
                  "PassOutcome", "PassLength", "AirYards", "PassLocation", "RushAttempt", \
                  "RunLocation", "Reception", "Season"]]

#creating a column to count play for future sumations
nfl["PlayCount"] = 1

#dropping rows where [down]=Nan; these represent special teams plays and extra points
nfl.dropna(subset=["down"], inplace=True)
nfl.count()

C:\Users\Kristopher Bognot\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Kristopher Bognot\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Date            346534
GameID          346534
down            346534
Yards.Gained    346534
FirstDown       346534
Touchdown       346534
PlayType        346534
PassOutcome     167724
PassLength      167163
AirYards        346534
PassLocation    167163
RushAttempt     346534
RunLocation     119329
Reception       346534
Season          346534
PlayCount       346534
dtype: int64

In [61]:
#NFL Passing Data
#Grabbing only the records where the PlayType = Pass from our NFL Data
nfl_pass = nfl.loc[nfl['PlayType'] == "Pass"]

#Create DataFrame for only Passing columns
npass = nfl_pass[["down", "Yards.Gained", "AirYards", "FirstDown", "Touchdown", "PlayCount"]]

#Group by Down
p = npass.groupby(["down"]).sum()
p = n.reset_index()

#Create DataFrame of Totals for all downs
totals = pd.DataFrame({"down": ["Totals"], 
                       "Yards.Gained": n["Yards.Gained"].sum(),
                       "AirYards": n["AirYards"].sum(),
                       "FirstDown": n["FirstDown"].sum(),
                       "Touchdown": n["Touchdown"].sum(),
                       "PlayCount": n["PlayCount"].sum()})

#Append Totals to Pass table
p = p.append(totals)
p

,down,Yards.Gained,AirYards,FirstDown,Touchdown,PlayCount
0,1,447898,512854,16030.0,2613,59329
1,2,367297,402372,16295.0,2419,52840
2,3,312234,387925,15568.0,2379,44229
3,4,15725,26602,2098.0,238,2530
0,Totals,1143154,1329753,49991.0,7649,158928
